In [ ]:
import matplotlib.pyplot as plt

: 

In [11]:
with open("names.txt", "r") as file:
    words = file.read()

words = words.splitlines()
print(f"Five first lines is: {words[:5]}")

Five first lines is: ['emma', 'olivia', 'ava', 'isabella', 'sophia']


In character language models some name (ie. Maya) is interpreted as:

- M is likely to come first ($P("M"|"")$ is higher than other letters)
- a is likely to come second if M is first ($P("a"|"M")$ is higher...)
- y is likely to come third if M is first and a is second ($P("y"|"Ma")$ is higher...)
- a is likely to come fourth if M is first, a is second and y is third ($P("a"|"May")$ is higher...)

## Bigram model

Bigrams are the simplest form of character language models. They are only concerned with consecutive pairs of letters. Each pair is called a bigram.

In [16]:
for word in words[:1]:
    for ch1, ch2 in zip(word, word[1:]):
        print(ch1, ch2)

e m
m m
m a


In order to understand where the start and end of a word is we need to add some special characters that will signalize that.

In [20]:
for word in words[:1]:
    w = ["<S>"] + list(word) + ["<E>"]
    for ch1, ch2 in zip(w, w[1:]):
        print(ch1, ch2)

<S> e
e m
m m
m a
a <E>


The simplest method to build a statistical method using bigrams is just to count them. The ```.get(key, value)``` is a dictionary metod to return a value of a given key or set a given optional value if the key is not found.

In [23]:
b = {}
for word in words:
    w = ["<S>"] + list(word) + ["<E>"]
    for ch1, ch2 in zip(w, w[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1